In [16]:
import os
import numpy as np
from datafilereader import DataFileReader
import pickle
import glob

# Path to the data folder containing the live files
folder_path = "./data/Live_files/" 

# Set up features
featureListR = [17, 20, 21, 22, 26, 28, 31, 32, 39, 42, 44, 71]
featureListL = [4, 5, 6, 7, 8, 9, 10, 12, 61]
labels = ["unknown", "5_CTS", "10_CTS", "20_CTS", "50_CTS", "1_CHF", "2_CHF", "5_CHF"]

# Load the pre-trained models
with open("data/Tests/modelAn1.pkl", "rb") as f:
    ANO = pickle.load(f)

with open("data/Tests/model1.pkl", "rb") as f:
    SVM = pickle.load(f)

# Count the number of files in the folder
def count_files_in_folder(folder_path):
    try:
        files = [item for item in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, item))]
        return len(files)
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0

# Process each test file
def process_files(folder_path):
    NmbFiles = count_files_in_folder(folder_path)
    if NmbFiles == 0:
        print("No files found in the folder.")
        return None, None
    else:
        print(f"Found {NmbFiles} files in the folder.")
    
    file_paths = glob.glob(os.path.join(folder_path, "test_set_*.h5"))
    
    summary_results = []
    detailed_results = {}

    for file_path in file_paths:
        # Get the number of the file
        file = file_path.split("_")[-1].split(".")[0]
        
        dataset = DataFileReader(file_path)
        
        # Get all measured Z for this coin
        f, Z = dataset.get_all_mesurements()
        R = np.real(Z)
        L = np.imag(Z) / (2 * np.pi * f)
        
        # Get the calibration index
        C_idx = dataset.get_reference_impedance_index()
        if C_idx is None or len(C_idx) == 0:
            print(f"No reference measurement found in {file_path}")
            continue
        
        R_cal = R[C_idx,:]
        L_cal = L[C_idx,:]
        
        # Remove the calibration index from the list of measurements
        R = np.delete(R, C_idx, axis=0)
        L = np.delete(L, C_idx, axis=0)
        
        # Calibrate the coin measurements
        R = R - R_cal
        L = L - L_cal
        # Extract the needed features
        R = R[:, featureListR]
        L = L[:, featureListL]
        
        # Ensure no zero or near-zero values in L to avoid division by zero
        L[L == 0] = np.nan  # Replace zero with NaN to handle safely
        p = R[:,0:9] / L
        
        # Collect results for each measurement
        results = []
        
        # Initialize prediction counter for the summary
        predictions = {label: 0 for label in labels}

        # Get the number of measurements after removing calibration
        N = R.shape[0]

        for j in range(N):
            # Concatenate the features
            X = np.concatenate((R[j, :], L[j, :], p[j, :]), axis=0)
            # Check for NaN values before prediction
            if np.isnan(X).any():
                results.append(("Measurement {}".format(j), "Skipped due to NaN"))
                continue
            
            # Reshape the array to 2D
            X = X.reshape(1, -1)
            
            # Anomaly detection
            try:
                result = ANO.predict(X)
            except ValueError as ve:
                results.append(("Measurement {}".format(j), "Anomaly detection error"))
                continue
            
            if result == -1:
                results.append(("Measurement {}".format(j), "Unknown"))
                predictions["unknown"] += 1
            else:
                # Classification
                try:
                    Y = SVM.predict(X)
                    results.append(("Measurement {}".format(j), labels[Y[0]]))
                    predictions[labels[Y[0]]] += 1
                except ValueError as ve:
                    results.append(("Measurement {}".format(j), "Classification error"))
                    continue
        
        # Convert results to NumPy array
        results_array = np.array(results, dtype=object)
        detailed_results[file] = results_array
        
        # Add summary for this file
        file_summary = {"file": file, "predictions": predictions}
        summary_results.append(file_summary)

    return summary_results, detailed_results

# Main function to process all test files
def main(folder_path):
    return process_files(folder_path)

# Run the main function and get the results
summary_results, detailed_results = main(folder_path)


c:\Users\Zack\miniconda3\envs\ML\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator ExtraTreeRegressor from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Zack\miniconda3\envs\ML\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator IsolationForest from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Zack\miniconda3\envs\ML\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.4.2 when using version 1.4.

Found 3 files in the folder.


In [17]:
# Cell to print summary results
print("\nSummary of Results for All Files:")
for summary in summary_results:
    print(f"Test Set N°{summary['file']}:")
    for coin, count in summary["predictions"].items():
        if count > 0:
            print(f"  {coin}: {count}")



Summary of Results for All Files:
Test Set N°1:
  10_CTS: 5
  1_CHF: 35
Test Set N°2:
  2_CHF: 30
Test Set N°3:
  2_CHF: 1
  5_CHF: 49


In [18]:
# Cell to print detailed results for each measurement
for file, results_array in detailed_results.items():
    print(f"\nDetailed Results for test set N°{file}:")
    print("{:<15} {:<20}".format("Measurement", "Coin Type"))
    print("-" * 35)
    for row in results_array:
        print("{:<15} {:<20}".format(row[0], row[1]))



Detailed Results for test set N°1:
Measurement     Coin Type           
-----------------------------------
Measurement 0   1_CHF               
Measurement 1   10_CTS              
Measurement 2   1_CHF               
Measurement 3   1_CHF               
Measurement 4   1_CHF               
Measurement 5   1_CHF               
Measurement 6   1_CHF               
Measurement 7   1_CHF               
Measurement 8   1_CHF               
Measurement 9   1_CHF               
Measurement 10  1_CHF               
Measurement 11  1_CHF               
Measurement 12  1_CHF               
Measurement 13  10_CTS              
Measurement 14  1_CHF               
Measurement 15  1_CHF               
Measurement 16  1_CHF               
Measurement 17  1_CHF               
Measurement 18  1_CHF               
Measurement 19  1_CHF               
Measurement 20  1_CHF               
Measurement 21  1_CHF               
Measurement 22  1_CHF               
Measurement 23  1_CHF               
Mea

In [1]:
import time
import os
import numpy as np
from datafilereader import DataFileReader
import pickle

# Path to the data file
file_path = "./data/Tests/coin_data.h5" 

#Set up features
featureListR = [17,20,21,22,26,28,31,32,39,42,44,71]
featureListL = [4,5,6,7,8,9,10,12,61]


labels = ["unknown", "5_CTS", "10_CTS", "20_CTS", "50_CTS", "1_CHF", "2_CHF", "5_CHF"]

# Load the pre-trained SVM model

with open("data/Tests/model1.pkl","rb+") as f:
    SVM = pickle.load(f)

with open("data/Tests/modelAn1.pkl","rb+") as f:
    ANO = pickle.load(f)


# Main function
def main(file_path):

    #SETUP
    Calibrated = False
    reader = DataFileReader(file_path)

    # CALIBRATION
    print("Please Start Calibration")
    while not Calibrated:
    
        if reader.has_file_changed():
            f, Z, is_reference = reader.get_last_mesurement()
            R_cal = np.real(Z)
            L_cal = np.imag(Z)/(2*np.pi*f)
            Calibrated = True
            print("Calibration done")
            

        time.sleep(2)  # Give user time to calibrate
        
    # TESTING
    print("Please take the measurement for the coin to be tested")
    while  Calibrated:
        if reader.has_file_changed():
            f, Z, is_reference = reader.get_last_mesurement()
            R = np.real(Z)
            L = np.imag(Z)/(2*np.pi*f)
            # calibrate the coin
            R = R-R_cal
            L = L-L_cal
            
            #extract the needed features
            R = R[featureListR]
            L = L[featureListL]
            p = R[0:9]/L
            #concatenate the features
            X = np.concatenate((R,L,p),axis=0)
            # reshape the array to 2D
            X = X.reshape(1, -1)
            result = ANO.predict(X)
            if result == -1:
                print("Coin is unknown")
            else:
                Y = SVM.predict(X)
                print("Coin is of type: ", labels[Y[0]])

        time.sleep(2)  # Give user time to mesure the coin to be tested
        
    


c:\Users\Zack\miniconda3\envs\ML\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Zack\miniconda3\envs\ML\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator ExtraTreeRegressor from version 1.4.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Zack\miniconda3\envs\ML\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator IsolationForest from version 1.4.2 when using version 1.4.

In [3]:
main(file_path)

Please Start Calibration


KeyboardInterrupt: 